# 如何将 BaseChatMessageHistory 与 LangGraph 结合使用

:::info 先决条件

本指南假设您熟悉以下概念：
* [聊天历史](/docs/concepts/chat_history)
* [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)
* [LangGraph](https://langchain-ai.github.io/langgraph/concepts/high_level/)
* [记忆](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#memory)
:::

我们建议新的 LangChain 应用程序利用 [内置的 LangGraph 持久化](https://langchain-ai.github.io/langgraph/concepts/persistence/) 来实现记忆。

在某些情况下，用户可能需要继续使用现有的聊天消息历史持久化解决方案。

在这里，我们将展示如何将 [LangChain 聊天消息历史记录](https://python.langchain.com/docs/integrations/memory/)（[BaseChatMessageHistory](https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html) 的实现）与 LangGraph 结合使用。

## 设置

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-anthropic langgraph

In [ ]:
import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

## ChatMessageHistory

消息历史需要通过会话 ID 或用户 ID 和会话 ID 的二元组进行参数化。

许多 [LangChain 聊天消息历史](https://python.langchain.com/docs/integrations/memory/) 都包含 `session_id` 或某个 `namespace`，以便跟踪不同的对话。请参考具体的实现来检查它是如何参数化的。

内置的 `InMemoryChatMessageHistory` 不包含此类参数化，因此我们将创建一个字典来跟踪消息历史。

In [ ]:
import uuid

from langchain_core.chat_history import InMemoryChatMessageHistory

chats_by_session_id = {}


def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    chat_history = chats_by_session_id.get(session_id)
    if chat_history is None:
        chat_history = InMemoryChatMessageHistory()
        chats_by_session_id[session_id] = chat_history
    return chat_history

## 配合 LangGraph 使用

接下来，我们将使用 LangGraph 来设置一个基础聊天机器人。如果你不熟悉 LangGraph，请查阅以下[快速入门教程](https://langchain-ai.github.io/langgraph/tutorials/introduction/)。

我们将为聊天模型创建一个[LangGraph 节点](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes)，并手动管理对话历史，同时考虑作为 RunnableConfig 的一部分传入的对话 ID。

对话 ID 可以作为 RunnableConfig 的一部分（就像我们在这里所做的那样）传入，也可以作为[图状态](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)的一部分传入。

In [6]:
import uuid

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
builder = StateGraph(state_schema=MessagesState)

# Define a chat model
model = ChatAnthropic(model="claude-3-haiku-20240307")


# Define the function that calls the model
def call_model(state: MessagesState, config: RunnableConfig) -> list[BaseMessage]:
    # Make sure that config is populated with the session id
    if "configurable" not in config or "session_id" not in config["configurable"]:
        raise ValueError(
            "Make sure that the config includes the following information: {'configurable': {'session_id': 'some_value'}}"
        )
    # Fetch the history of messages and append to it any new messages.
    # highlight-start
    chat_history = get_chat_history(config["configurable"]["session_id"])
    messages = list(chat_history.messages) + state["messages"]
    # highlight-end
    ai_message = model.invoke(messages)
    # Finally, update the chat message history to include
    # the new input message from the user together with the
    # response from the model.
    # highlight-next-line
    chat_history.add_messages(state["messages"] + [ai_message])
    return {"messages": ai_message}


# Define the two nodes we will cycle between
builder.add_edge(START, "model")
builder.add_node("model", call_model)

graph = builder.compile()

# Here, we'll create a unique session ID to identify the conversation
session_id = uuid.uuid4()
config = {"configurable": {"session_id": session_id}}

input_message = HumanMessage(content="hi! I'm bob")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
input_message = HumanMessage(content="what was my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! It's nice to meet you. I'm Claude, an AI assistant created by Anthropic. How are you doing today?
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

You introduced yourself as Bob when you said "hi! I'm bob".


:::tip

如果使用 langgraph >= 0.2.28，这也支持逐个 token 流式传输 LLM 内容。
:::

In [3]:
from langchain_core.messages import AIMessageChunk

first = True

for msg, metadata in graph.stream(
    {"messages": input_message}, config, stream_mode="messages"
):
    if msg.content and not isinstance(msg, HumanMessage):
        print(msg.content, end="|", flush=True)

You| sai|d your| name was Bob.|

## 使用 RunnableWithMessageHistory

此操作指南直接使用了 `BaseChatMessageHistory` 的 `messages` 和 `add_messages` 接口。

另外，你可以使用 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)，因为 [LCEL](/docs/concepts/lcel/) 可以在任何 [LangGraph 节点](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes) 中使用。

要做到这一点，请替换以下代码：

```python
def call_model(state: MessagesState, config: RunnableConfig) -> list[BaseMessage]:
    # highlight-start
    # 确保 config 已填充了 session id
    if "configurable" not in config or "session_id" not in config["configurable"]:
        raise ValueError(
            "您需要确保 config 包含以下信息：{'configurable': {'session_id': 'some_value'}}"
        )
    # 获取消息历史，并将任何新消息追加到其中。
    chat_history = get_chat_history(config["configurable"]["session_id"])
    messages = list(chat_history.messages) + state["messages"]
    ai_message = model.invoke(messages)
    # 最后，更新聊天消息历史，包含
    # 用户的新输入消息以及
    # 模型的响应。
    chat_history.add_messages(state["messages"] + [ai_message])
    # hilight-end
    return {"messages": ai_message}
```

替换为在你当前应用程序中定义的 `RunnableWithMessageHistory` 的相应实例。

```python
runnable = RunnableWithMessageHistory(...) # 来自现有代码

def call_model(state: MessagesState, config: RunnableConfig) -> list[BaseMessage]:
    # RunnableWithMessageHistory 负责读取消息历史
    # 并用新的人类消息和 AI 响应进行更新。
    ai_message = runnable.invoke(state['messages'], config)
    return {
        "messages": ai_message
    }
```